# Different sources have different prices for the same currency pair

Some sources look at currency futures, some at spot. Currencies are traded 24/7 so depending on the time at which a "close" is computed, you could land up with different prices.

pluto has 3 different sources for USDINR: Fred, Nse Futures and AlphaVantage. Here, we will look at how differ from one-another.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(ggpubr)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
currencies <- Currencies()
fred<-Fred()

In [ ]:
# sample: https://github.com/shyams80/plutons/blob/master/docs-R/Currencies.ipynb
# from the sample, we know that USDINR started trading on the NSE from 2010-10-29 
# but AlphaVantage data goes back only to 2014-11-07

# sample: http://pluto.studio/user/shyams80/notebooks/docs-R/Fred.ipynb, Fred series is DEXINUS -2147478748
startDate <- as.Date("2014-11-07")

fredFx <- fred$TimeSeries() %>% 
    filter(SERIES_ID == -2147478748, TIME_STAMP >= startDate) %>% 
    select(TIME_STAMP, VAL) %>%
    collect()

closestExpiries <- currencies$NseFuturesTimeSeries() %>%
    filter(TIME_STAMP >= startDate & SYMBOL == 'USDINR') %>%
    group_by(TIME_STAMP) %>%
    summarize(EXPIRY_DT = min(EXPIRY)) 

nseFx <- closestExpiries %>% 
    inner_join(currencies$NseFuturesTimeSeries(), by=c('TIME_STAMP' = 'TIME_STAMP', 'EXPIRY_DT' = 'EXPIRY')) %>%
    filter(TIME_STAMP >= startDate & SYMBOL == 'USDINR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

avFx <- currencies$AvEodTimeSeries() %>%
    filter(TIME_STAMP >= startDate & SYMBOL == 'INR') %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

In [ ]:
monthlies <- merge(Common.NormalizeMonthlyDates(monthlyReturn(xts(fredFx$VAL, fredFx$TIME_STAMP))), 
                   Common.NormalizeMonthlyDates(monthlyReturn(xts(nseFx$CLOSE, nseFx$TIME_STAMP))),
                   Common.NormalizeMonthlyDates(monthlyReturn(xts(avFx$CLOSE, avFx$TIME_STAMP))))
names(monthlies) <-c('FRED', 'NSE', "AV")

monthlies <- monthlies[-1,]
monthlies <- monthlies[-nrow(monthlies),]

print(head(monthlies))

In [ ]:
Common.PlotCumReturns(monthlies, "USDINR from different sources", NA)